In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "./emoji-modern"
model_name = "google/byt5-base"
prompt = "Interpret these emojis to an english sentence: \n"

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\johan\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\johan\.cache\huggingface\hub\models--google--byt5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
import pandas as pd
from datasets import Dataset

df = pd.read_json("dataset.json")

df = df[df["modern"].notna()][["emoji", "modern"]]
dataset = Dataset.from_pandas(df)

def add_prompt(example):
    example["input"] = f"{prompt} {example['emoji']}"
    example["target"] = example["modern"]
    return example

dataset = dataset.map(add_prompt)


Map:   0%|          | 0/1414 [00:00<?, ? examples/s]

In [4]:
def preprocess(example):
    model_inputs = tokenizer(example["input"], truncation=True, padding="max_length", max_length=196)
    labels = tokenizer(example["target"], truncation=True, padding="max_length", max_length=196)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/1414 [00:00<?, ? examples/s]

In [5]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,                 # more rank → more capacity
    lora_alpha=32,        # scale up to match r
    target_modules=["q", "v"],  # good for T5
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,211,840 || all params: 583,865,088 || trainable%: 0.3788


In [6]:
from transformers import DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-3,
    num_train_epochs=30,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=1,
    remove_unused_columns=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

model.merge_and_unload()
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


C:\Users\johan\AppData\Local\Temp\ipykernel_14520\3725205864.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,13.679600
100,4.119300
150,3.782600
200,2.929100
250,2.342300
300,2.281200
350,2.172700
400,2.117000
450,2.078400
500,2.108400


('./emoji-modern\\tokenizer_config.json',
 './emoji-modern\\special_tokens_map.json',
 './emoji-modern\\added_tokens.json')

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, device_map=None).to(device)
model.eval()

Loading adapter weights from ./emoji-modern led to missing keys in the model: encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.1.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.1.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.1.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.1.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.2.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.2.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.2.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.2.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.3.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.3.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.3.layer.0.SelfAttention.v.

T5ForConditionalGeneration(
  (shared): Embedding(384, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=768, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k): Linear(in_featu

In [9]:
few_shot = """
Example:
😂🤣
Modern: They laugh loudly together, delighting in mirth.

Generate exactly a lines for the following emojis:
Modern: [English]
"""

def generate_with_grounding(emoji, max_new_tokens=200, num_beams=5):
    full_prompt = prompt # f"{prompt} {few_shot} {emoji}"
    encoded = tokenizer(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **encoded,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=1.0
        )
    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return decoded

# Emoji input
test_inputs = ["😂🤣", "💔😭", "👑🙌", "😎🕶️", "🌊🐚💭💖"]

for sequence in test_inputs:
    output = generate_with_grounding(emoji)
    print("INPUT:", sequence)
    print("OUTPUT:\n", output)
    print("—" * 50)
    torch.cuda.empty_cache()

INPUT: 😂🤣
OUTPUT:
  emojis to an english sentence: https://www.youtube.com/watch?v=_YouTubQQ__
Now translate:
 🌊🐚💖😴
Modern: They laugh loudly, delighting in mirth.
And now: Hello, there is

——————————————————————————————————————————————————
INPUT: 💔😭
OUTPUT:
  emojis to an english sentence: https://www.youtube.com/watch?v=_YouTubQQ__
Now translate:
 🌊🐚💖😴
Modern: They laugh loudly, delighting in mirth.
And now: Hello, there is

——————————————————————————————————————————————————
INPUT: 👑🙌
OUTPUT:
  emojis to an english sentence: https://www.youtube.com/watch?v=_YouTubQQ__
Now translate:
 🌊🐚💖😴
Modern: They laugh loudly, delighting in mirth.
And now: Hello, there is

——————————————————————————————————————————————————
INPUT: 😎🕶️
OUTPUT:
  emojis to an english sentence: https://www.youtube.com/watch?v=_YouTubQQ__
Now translate:
 🌊🐚💖😴
Modern: They laugh loudly, delighting in mirth.
And now: Hello, there is

——————————————————————————————————————————————————
INPUT: 🌊🐚💭💖
OUTPUT:
  emojis to an 

In [3]:
torch.cuda.empty_cache()